In [44]:
import pandas as pd
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
import multiprocessing
from numpy import mean
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

df_cons_SL = pd.read_csv("df_cons_SL_NaN-incl.csv")

# remove insatnces with missing values in target class
df_cons_SL = df_cons_SL.dropna(subset=['lipophylic_toxins'], how="any").copy()

X = df_cons_SL.copy().drop("lipophylic_toxins", axis=1)
y = df_cons_SL["lipophylic_toxins"]

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)
# print(f"Class labels pre-transform: {list(le.classes_)}")
y = le.transform(y)
print(f"class encoding: ['neg','poz'] -> {le.transform(['neg','poz'])}")

class encoding: ['neg','poz'] -> [0 1]


In [ ]:
pipeline = Pipeline([
   ('imp', SimpleImputer()),
    ('norm', StandardScaler()),
   # ('norm', RobustScaler()),
    ('over', SMOTE(sampling_strategy=0.5)),
    ('under', RandomUnderSampler(sampling_strategy=0.8)),
    ('clf', RandomForestClassifier())
])

parameters = {
              'over__k_neighbors': range(1,11),
              'clf__n_estimators': [50,100,150],
              'clf__criterion': ['gini', 'entropy'],
              'clf__class_weight': ['balanced', 'balanced_subsample', None],
         
}
nfolds = 3
scores = ['recall', 'f1', 'roc_auc']
gscv = GridSearchCV(pipeline, 
                    parameters, 
                    scoring=scores,
                    cv=nfolds, #RepeatedStratifiedKFold(n_splits=3, n_repeats=1),
                    n_jobs=int(multiprocessing.cpu_count()/2), 
                    return_train_score=False, 
                    verbose=1, 
                    refit=False)
resultsGSCV = gscv.fit(X, y)
results = pd.DataFrame(resultsGSCV.cv_results_)
display(results.sort_values(by=[f'rank_test_recall']).transpose())

Fitting 3 folds for each of 180 candidates, totalling 540 fits
